In [15]:
import numpy as npn
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import math
import datetime
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/xxiang27/sikkaAIHackathon/main/data_file.csv')
df.head()

,id,month,year,visits,no_of_appts,production
0,34,3,2014,1,960,76.55
1,34,4,2014,488,1089,106333.80
2,34,5,2014,831,993,192230.40
3,34,6,2014,753,884,163515.12
4,34,7,2014,721,797,160185.90


In [3]:
df['day'] = 1
dates = pd.to_datetime(df[['month', 'year','day']].copy())
dates.head()

0   2014-03-01
1   2014-04-01
2   2014-05-01
3   2014-06-01
4   2014-07-01
dtype: datetime64[ns]

In [4]:
def turntoDate(df):
  lst = []
  for i in range(len(df)):
    lst.append(pd.Timestamp(year = df.loc[i, 'year'], month = df.loc[i, 'month'], day=None))
  df['date'] = lst
  return df

In [5]:
df_final = pd.concat([dates, df],axis=1)
df_final = df_final.drop(columns = ['month', 'year', 'day'])
df_final.head()

,0,id,visits,no_of_appts,production
0,2014-03-01,34,1,960,76.55
1,2014-04-01,34,488,1089,106333.80
2,2014-05-01,34,831,993,192230.40
3,2014-06-01,34,753,884,163515.12
4,2014-07-01,34,721,797,160185.90


In [6]:
# separate dataframe into separate dataframes for each practice 

import math 

def separate(df):
  '''takes a dataframe returns a list of dataframes, 
  one for each unique value under the id column'''
  lst = []
  grouped = df.groupby(df.id)
  for i in range(1, 285):
    lst.append(grouped.get_group(i))
  return lst 

def train_test_split(lst, add_test):
  '''takes a list of dataframes and returns the time series train and test split based on the percent'''
  test_set = []
  for df in lst:
    #add_test = math.floor(len(df) * percent)
    #df_add = df.tail(add_test)
    df_add = df.tail(add_test).copy()
    test_set.append(df_add)
    df.drop(df.tail(add_test).index, inplace = True)
  df_train = pd.concat(lst, ignore_index=True)
  df_test = pd.concat(test_set, ignore_index = True)
  return df_train, df_test
  

In [7]:
train_lst = separate(df_final)
df_train, df_test = train_test_split(train_lst, 4)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
first = df_train.loc[df_train['id'] ==  34]
first.head()

,0,id,visits,no_of_appts,production
3034,2014-03-01,34,1,960,76.55
3035,2014-04-01,34,488,1089,106333.80
3036,2014-05-01,34,831,993,192230.40
3037,2014-06-01,34,753,884,163515.12
3038,2014-07-01,34,721,797,160185.90


In [9]:
first.set_index(0, inplace=True)
first.head()

,id,visits,no_of_appts,production
0,,,,
2014-03-01,34,1,960,76.55
2014-04-01,34,488,1089,106333.80
2014-05-01,34,831,993,192230.40
2014-06-01,34,753,884,163515.12
2014-07-01,34,721,797,160185.90


In [10]:
Y= first.drop(columns = ['id', 'visits', 'no_of_appts'])
Y.head()

,production
0,
2014-03-01,76.55
2014-04-01,106333.80
2014-05-01,192230.40
2014-06-01,163515.12
2014-07-01,160185.90


In [11]:
X = first.drop(columns = ['id','production', 'no_of_appts'])

In [12]:
my_order = (1, 1, 1)
my_seasonal_order = (1, 1, 1, 12)
model = sm.tsa.statespace.SARIMAX(Y,order = my_order, seasonal_order = my_seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
model_fit = model.fit()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)


In [22]:
yhat = model_fit.predict(start = len(first), end = len(first)+3)

In [23]:
yhat

2020-09-01    233249.687075
2020-10-01    245999.427856
2020-11-01    204808.534201
2020-12-01    216614.155380
Freq: MS, dtype: float64

In [21]:
first

,id,visits,no_of_appts,production
0,,,,
2014-03-01,34,1,960,76.55
2014-04-01,34,488,1089,106333.80
2014-05-01,34,831,993,192230.40
2014-06-01,34,753,884,163515.12
2014-07-01,34,721,797,160185.90
...,...,...,...,...
2020-04-01,34,11,466,2164.32
2020-05-01,34,119,438,22164.32
2020-06-01,34,605,706,160963.22
